In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/Data_Klaim.csv
/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/sample_submission.csv
/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/Data_Polis.csv


# DATA FOUNDATION

In [1]:
# ============================================================
# STAGE 1 v5 — FOUNDATION (LB-GRADE AUDITABLE)
# Key upgrades:
# - Multi date-mode: build year_month from service/payment/other tanggal columns
# - No month gets dropped because severity/rate NaN (safe fill + eps logs)
# - Exposure: claimant + inforce (audit + fallback) without hiding issues
# ============================================================

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

BASE_PATH = "/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/"
klaim = pd.read_csv(BASE_PATH + "Data_Klaim.csv")
polis = pd.read_csv(BASE_PATH + "Data_Polis.csv")

# =============================
# CLEAN COLUMN NAMES
# =============================
def clean_columns(df):
    df = df.copy()
    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace("/", "_", regex=False)
        .str.replace("-", "_", regex=False)
    )
    return df

klaim = clean_columns(klaim)
polis = clean_columns(polis)

# =============================
# DATE PARSING (YYYYMMDD int + dd/mm/yyyy + general)
# =============================
def parse_mixed_date(s: pd.Series) -> pd.Series:
    s = s.copy()
    idx = s.index

    if pd.api.types.is_numeric_dtype(s):
        ss = s.astype("Int64").astype(str)
    else:
        ss = s.astype(str).str.strip()

    ss = ss.replace({"<NA>": np.nan, "nan": np.nan, "None": np.nan, "NaT": np.nan})

    out = pd.Series(pd.NaT, index=idx, dtype="datetime64[ns]")

    m8 = ss.str.fullmatch(r"\d{8}", na=False)
    if m8.any():
        out.loc[m8] = pd.to_datetime(ss.loc[m8], format="%Y%m%d", errors="coerce")

    rest = ~m8 & ss.notna()
    if rest.any():
        has_slash = ss.loc[rest].str.contains("/", na=False)
        if has_slash.any():
            out.loc[ss.loc[rest][has_slash].index] = pd.to_datetime(
                ss.loc[rest][has_slash], errors="coerce", dayfirst=True
            )
        if (~has_slash).any():
            out.loc[ss.loc[rest][~has_slash].index] = pd.to_datetime(
                ss.loc[rest][~has_slash], errors="coerce"
            )
    return out

for col in klaim.columns:
    if "tanggal" in col:
        klaim[col] = parse_mixed_date(klaim[col])

for col in polis.columns:
    if "tanggal" in col:
        polis[col] = parse_mixed_date(polis[col])

# =============================
# SAFE DEDUP
# =============================
claim_id_col = None
for c in ["claim_id", "id_klaim", "klaim_id"]:
    if c in klaim.columns:
        claim_id_col = c
        break

if claim_id_col is not None:
    klaim = klaim.drop_duplicates(subset=[claim_id_col]).reset_index(drop=True)
else:
    klaim = klaim.drop_duplicates().reset_index(drop=True)

if "nomor_polis" in polis.columns:
    polis = polis.drop_duplicates(subset=["nomor_polis"]).reset_index(drop=True)
else:
    polis = polis.drop_duplicates().reset_index(drop=True)

# =============================
# NOMINAL COLUMN (RAW TARGET)
# =============================
nom_col = "nominal_klaim_yang_disetujui"
if nom_col not in klaim.columns:
    cand = [c for c in klaim.columns if "nominal" in c]
    if len(cand) == 0:
        raise ValueError("No nominal column found in klaim.")
    nom_col = cand[0]

raw_nom = pd.to_numeric(klaim[nom_col], errors="coerce").fillna(0.0)
raw_nom = raw_nom.clip(lower=0.0)
klaim[nom_col] = raw_nom

# Optional winsor copy (features only)
klaim["nominal_klaim_clip"] = raw_nom.copy()
pos = klaim["nominal_klaim_clip"] > 0
if pos.any():
    low_q  = klaim.loc[pos, "nominal_klaim_clip"].quantile(0.005)
    high_q = klaim.loc[pos, "nominal_klaim_clip"].quantile(0.995)
    klaim.loc[pos, "nominal_klaim_clip"] = klaim.loc[pos, "nominal_klaim_clip"].clip(low_q, high_q)

# =============================
# MERGE
# =============================
if "nomor_polis" not in klaim.columns:
    raise ValueError("klaim tidak punya kolom nomor_polis.")
df0 = klaim.merge(polis, on="nomor_polis", how="left")

# =============================
# DATE MODE: build multiple year_month candidates
# =============================
# Kamu boleh ganti prioritas kandidat ini sesuai kolom real di datamu.
date_candidates = []
# Prefer "service" style
for c in ["tanggal_pasien_masuk_rs", "tanggal_masuk_rs", "tanggal_perawatan"]:
    if c in df0.columns:
        date_candidates.append(("service", c))
        break

# Prefer "payment" style
for c in ["tanggal_pembayaran_klaim", "tanggal_bayar", "tanggal_pembayaran"]:
    if c in df0.columns:
        date_candidates.append(("payment", c))
        break

# Fallback: semua tanggal lain di klaim (urutkan stabil)
other_tcols = [c for c in df0.columns if ("tanggal" in c)]
for c in other_tcols:
    if c not in [x[1] for x in date_candidates]:
        date_candidates.append((f"other:{c}", c))

if len(date_candidates) == 0:
    raise ValueError("Tidak ada kolom tanggal untuk membangun year_month.")

print("Date candidates:", date_candidates[:6], "..." if len(date_candidates) > 6 else "")

# =============================
# EXPOSURE: claimant + inforce (audit friendly)
# =============================
start_col = None
for c in ["tanggal_efektif_polis", "tanggal_mulai_polis", "tanggal_mulai"]:
    if c in polis.columns:
        start_col = c
        break

def build_exposure_tables(df_mode: pd.DataFrame, year_month_col: str):
    # complete months
    ym = df_mode[year_month_col].dropna()
    if ym.empty:
        raise ValueError("year_month kosong pada mode ini.")
    min_m = ym.min()
    max_m = ym.max()
    all_months = pd.period_range(min_m, max_m, freq="M")

    # claimant exposure: unique policies claiming in month
    expo_claimant = (
        df_mode.groupby(year_month_col)["nomor_polis"].nunique()
              .reindex(all_months, fill_value=0)
              .rename("exposure_claimant")
              .rename_axis("year_month")
              .reset_index()
    )

    # inforce exposure: cumulative started policies (no end-date)
    if start_col is not None:
        p = polis[["nomor_polis", start_col]].dropna(subset=[start_col]).copy()
        p["start_m"] = p[start_col].dt.to_period("M")

        # number of unique policies that have started by each month
        # (approx inforce without lapse)
        started_by_m = (
            p.groupby("start_m")["nomor_polis"].nunique()
             .reindex(all_months, fill_value=0)
             .cumsum()
        )
        # add policies that started before min_m
        base = p.loc[p["start_m"] < min_m, "nomor_polis"].nunique()
        expo_inforce = (base + started_by_m).rename("exposure_inforce").reset_index().rename(columns={"start_m":"year_month"})
        expo_inforce = expo_inforce.rename(columns={"year_month":"year_month"})
        # ensure key name matches
        expo_inforce = expo_inforce.rename(columns={"start_m":"year_month"})
        if "year_month" not in expo_inforce.columns:
            expo_inforce = expo_inforce.rename(columns={"index":"year_month"})
    else:
        expo_inforce = expo_claimant[["year_month"]].copy()
        expo_inforce["exposure_inforce"] = 0

    expo = expo_claimant.merge(expo_inforce, on="year_month", how="left")
    return expo, all_months

# =============================
# BUILD monthly for each date mode (so you can LB-test quickly)
# =============================
EPS = 1e-12
monthly_by_mode = {}
df_by_mode = {}

for mode, dcol in date_candidates:
    tmp = df0.copy()
    tmp = tmp.dropna(subset=["nomor_polis", dcol]).copy()
    tmp["year_month"] = tmp[dcol].dt.to_period("M")

    if tmp["year_month"].dropna().empty:
        continue

    expo, all_months = build_exposure_tables(tmp, "year_month")

    freq_col = claim_id_col if claim_id_col is not None and claim_id_col in tmp.columns else "nomor_polis"
    monthly_core = (
        tmp.groupby("year_month")
           .agg(
               frequency=(freq_col, "count"),
               total_claim=(nom_col, "sum")
           )
           .reindex(all_months, fill_value=0.0)
           .rename_axis("year_month")
           .reset_index()
    )

    monthly = monthly_core.merge(expo, on="year_month", how="left")

    # exposure choice (do NOT hide with ffill/bfill at this stage)
    # we keep both; later stages can choose
    # but provide a default exposure that is safest:
    # if inforce is mostly zeros -> fallback to claimant
    inforce_sum = float(np.nansum(monthly["exposure_inforce"].values))
    claimant_sum = float(np.nansum(monthly["exposure_claimant"].values))
    if inforce_sum > 0:
        monthly["exposure"] = monthly["exposure_inforce"].astype(float)
        exposure_mode_used = "inforce"
    else:
        monthly["exposure"] = monthly["exposure_claimant"].astype(float)
        exposure_mode_used = "claimant"

    # Targets & derived series (safe for zeros; no month dropped)
    monthly["frequency"] = monthly["frequency"].astype(float)
    monthly["total_claim"] = monthly["total_claim"].astype(float)

    monthly["severity"] = np.where(monthly["frequency"] > 0, monthly["total_claim"] / monthly["frequency"], 0.0)
    monthly["claim_rate"] = np.where(monthly["exposure"] > 0, monthly["frequency"] / monthly["exposure"], 0.0)

    # Logs (safe)
    monthly["log_total"] = np.log1p(np.clip(monthly["total_claim"], 0.0, np.inf))
    monthly["log_freq"]  = np.log1p(np.clip(monthly["frequency"], 0.0, np.inf))
    monthly["log_sev"]   = np.log1p(np.clip(monthly["severity"], 0.0, np.inf))
    monthly["log_rate"]  = np.log1p(np.clip(monthly["claim_rate"], 0.0, np.inf))

    # Time features
    monthly["month"] = monthly["year_month"].dt.month
    monthly["month_sin"] = np.sin(2*np.pi*monthly["month"]/12)
    monthly["month_cos"] = np.cos(2*np.pi*monthly["month"]/12)
    monthly["month_index"] = np.arange(len(monthly), dtype=int)

    # Volatility (safe)
    monthly["roll6"] = monthly["total_claim"].rolling(6, min_periods=3).mean()
    monthly["std6"]  = monthly["total_claim"].rolling(6, min_periods=3).std()
    monthly["vol_ratio"] = monthly["std6"] / np.where(monthly["roll6"] > 0, monthly["roll6"], np.nan)
    med = np.nanmedian(monthly["vol_ratio"].values)
    monthly["high_vol_regime"] = (monthly["vol_ratio"] > med).astype(int)

    # Lags (keep NaN; downstream stages decide how to fill)
    for col in ["log_total", "log_freq", "log_sev", "log_rate"]:
        monthly[f"{col}_lag1"] = monthly[col].shift(1)
        monthly[f"{col}_lag2"] = monthly[col].shift(2)
        monthly[f"{col}_lag3"] = monthly[col].shift(3)
        monthly[f"{col}_roll3"] = monthly[col].shift(1).rolling(3).mean()

    monthly_by_mode[mode] = (monthly, exposure_mode_used, dcol)
    df_by_mode[mode] = tmp

# =============================
# Choose default mode (you can swap later)
# =============================
DEFAULT_MODE = "service" if "service" in monthly_by_mode else list(monthly_by_mode.keys())[0]
monthly, EXPOSURE_MODE_USED, SERVICE_COL_USED = monthly_by_mode[DEFAULT_MODE]

df = df_by_mode[DEFAULT_MODE].copy()
df["active_policies"] = df.merge(
    monthly[["year_month", "exposure"]], on="year_month", how="left"
)["exposure"].values

print("\n==============================")
print("DEFAULT_MODE:", DEFAULT_MODE, "| DATE_COL:", SERVICE_COL_USED, "| EXPOSURE_USED:", EXPOSURE_MODE_USED)
print("Monthly shape:", monthly.shape, "| Unique months:", monthly["year_month"].nunique())
print("Freq min/max:", float(monthly["frequency"].min()), float(monthly["frequency"].max()))
print("Total min/max:", float(monthly["total_claim"].min()), float(monthly["total_claim"].max()))
print("Exposure (used) min/max:", float(monthly["exposure"].min()), float(monthly["exposure"].max()))
print("Inforce sum:", float(np.nansum(monthly["exposure_inforce"].values)), "| Claimant sum:", float(np.nansum(monthly["exposure_claimant"].values)))
print("==============================")
print("STAGE 1 v5 — READY (monthly_by_mode available)")
print("Available modes:", list(monthly_by_mode.keys()))

Date candidates: [('service', 'tanggal_pasien_masuk_rs'), ('payment', 'tanggal_pembayaran_klaim'), ('other:tanggal_pasien_keluar_rs', 'tanggal_pasien_keluar_rs'), ('other:tanggal_lahir', 'tanggal_lahir'), ('other:tanggal_efektif_polis', 'tanggal_efektif_polis')] 

DEFAULT_MODE: service | DATE_COL: tanggal_pasien_masuk_rs | EXPOSURE_USED: inforce
Monthly shape: (19, 36) | Unique months: 19
Freq min/max: 208.0 302.0
Total min/max: 9610379678.55 20260981490.193146
Exposure (used) min/max: 4096.0 4096.0
Inforce sum: 77824.0 | Claimant sum: 2671.0
STAGE 1 v5 — READY (monthly_by_mode available)
Available modes: ['service', 'payment', 'other:tanggal_pasien_keluar_rs', 'other:tanggal_lahir', 'other:tanggal_efektif_polis']


# TIME-SERIES DATASET ENGINEERING

In [4]:
import numpy as np
import pandas as pd

# ============================================================
# 🔹 BASIC CHECK
# ============================================================

assert "monthly" in globals(), "Stage 1 belum dijalankan."
assert "df" in globals(), "df tidak tersedia dari Stage 1."

all_months = monthly["year_month"].unique()
portfolio_expo = monthly[["year_month","exposure"]].copy()
portfolio_sev  = monthly[["year_month","severity"]].copy()

# ============================================================
# 🔹 SEGMENT DEFINITIONS (STABLE BUCKETING)
# ============================================================

# Care Type
if "care_type" not in df.columns:
    if "inpatient_outpatient" in df.columns:
        df["care_type"] = (
            df["inpatient_outpatient"]
            .astype(str)
            .str.upper()
            .str.strip()
        )
    else:
        df["care_type"] = "UNKNOWN"
df["care_type"] = df["care_type"].fillna("UNKNOWN")

# Cashless
if "is_cashless" not in df.columns:
    if "reimburse_cashless" in df.columns:
        rc = df["reimburse_cashless"].astype(str).str.upper().str.strip()
        df["is_cashless"] = rc.eq("C").astype(int)
    else:
        df["is_cashless"] = 0

# RS bucket
if "rs_bucket" not in df.columns:
    if "lokasi_rs" in df.columns:
        loc = df["lokasi_rs"].astype(str).str.upper().str.strip()
        df["rs_bucket"] = np.select(
            [
                loc.eq("INDONESIA"),
                loc.eq("SINGAPORE"),
                loc.eq("MALAYSIA")
            ],
            ["ID","SG","MY"],
            default="OTHER"
        )
    else:
        df["rs_bucket"] = "OTHER"
df["rs_bucket"] = df["rs_bucket"].fillna("OTHER")

# Plan Code Bucketing (keep top 8 only)
if "plan_code" not in df.columns:
    df["plan_code"] = "UNKNOWN"

top_plans = (
    df["plan_code"]
    .value_counts()
    .head(8)
    .index
)

df["plan_code"] = np.where(
    df["plan_code"].isin(top_plans),
    df["plan_code"],
    "OTHER"
)

seg_cols = ["plan_code","care_type","is_cashless","rs_bucket"]

# ============================================================
# 🔹 RAW SEGMENT AGGREGATION
# ============================================================

seg_raw = (
    df.groupby(["year_month"] + seg_cols)
      .agg(
          frequency=("nomor_polis","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum"),
          uniq_polis=("nomor_polis","nunique")
      )
      .reset_index()
)

# ============================================================
# 🔹 COMPLETE GRID (NO MISSING MONTHS)
# ============================================================

unique_segments = seg_raw[seg_cols].drop_duplicates()

grid = (
    unique_segments.assign(key=1)
    .merge(pd.DataFrame({"year_month": all_months, "key":1}), on="key")
    .drop("key",axis=1)
)

seg_full = (
    grid.merge(seg_raw, on=["year_month"]+seg_cols, how="left")
        .fillna(0)
)

# ============================================================
# 🔹 ROLLING SHARE (STABLE EXPOSURE ALLOCATION)
# ============================================================

# Rolling share based on uniq policies (3 month rolling)
seg_full = seg_full.sort_values(seg_cols + ["year_month"])

seg_full["rolling_uniq"] = (
    seg_full.groupby(seg_cols)["uniq_polis"]
    .transform(lambda x: x.shift(1).rolling(3,min_periods=1).mean())
)

# Normalize share per month
seg_full["share"] = (
    seg_full["rolling_uniq"] /
    seg_full.groupby("year_month")["rolling_uniq"].transform("sum")
).fillna(0)

# Merge portfolio exposure
seg_full = seg_full.merge(portfolio_expo, on="year_month", how="left")

seg_full["expo_seg"] = (
    seg_full["share"] * seg_full["exposure"]
).clip(lower=1.0)

# ============================================================
# 🔹 SEVERITY SHRINK (ANTI MICRO SEGMENT)
# ============================================================

seg_full = seg_full.merge(portfolio_sev, on="year_month", how="left", suffixes=("","_port"))

seg_full["sev_raw"] = np.where(
    seg_full["frequency"] > 0,
    seg_full["total_claim"] / seg_full["frequency"],
    0
)

# shrink factor based on frequency size
seg_full["alpha"] = (
    seg_full["frequency"] /
    (seg_full["frequency"] + 20)
)

seg_full["sev_shrunk"] = (
    seg_full["alpha"] * seg_full["sev_raw"] +
    (1 - seg_full["alpha"]) * seg_full["severity"]
)

# ============================================================
# 🔹 DERIVED METRICS
# ============================================================

seg_full["rate_seg"] = (
    seg_full["frequency"] /
    seg_full["expo_seg"]
)

seg_full["log_total"] = np.log1p(seg_full["total_claim"])
seg_full["log_freq"]  = np.log1p(seg_full["frequency"])
seg_full["log_sev"]   = np.log1p(seg_full["sev_shrunk"])
seg_full["log_rate"]  = np.log1p(seg_full["rate_seg"])

# Calendar
seg_full["month"] = seg_full["year_month"].dt.month
seg_full["month_sin"] = np.sin(2*np.pi*seg_full["month"]/12)
seg_full["month_cos"] = np.cos(2*np.pi*seg_full["month"]/12)

# Lags (strict no leakage)
for col in ["log_total","log_freq","log_sev","log_rate"]:
    seg_full[f"{col}_lag1"] = \
        seg_full.groupby(seg_cols)[col].shift(1)

    seg_full[f"{col}_lag2"] = \
        seg_full.groupby(seg_cols)[col].shift(2)

    seg_full[f"{col}_lag3"] = \
        seg_full.groupby(seg_cols)[col].shift(3)

# ============================================================
# 🔹 SAFE TRAIN PANEL
# ============================================================

seg_model = seg_full[
    seg_full["log_total_lag3"].notna()
].reset_index(drop=True)

# DO NOT fill everything with zero blindly
seg_model = seg_model.fillna(0)

# ============================================================
# FINAL CHECK
# ============================================================

print("SEG PANEL SHAPE:", seg_model.shape)
print("Unique segments:", seg_model[seg_cols].drop_duplicates().shape[0])
print("Months:", seg_model["year_month"].nunique())
print("Sum expo check (sample month):")

sample_month = seg_model["year_month"].iloc[-1]
print(
    "Segment expo sum:",
    seg_model[seg_model["year_month"]==sample_month]["expo_seg"].sum(),
    "| Portfolio expo:",
    monthly.loc[monthly["year_month"]==sample_month,"exposure"].values[0]
)

print("\nSTAGE 2 v2 — ELITE SEGMENT PANEL READY")

COMPACT PANEL SHAPE: (414, 29)
Unique segments: 41
Columns: 29

STAGE 2 — ELITE SEGMENT PANEL READY


# MODEL DEVELOPMENT

In [5]:
# ============================================================
# STAGE 3 v18 — SEVERITY-FIRST + RATE MODEL
# - Model claim_rate & severity directly
# - Frequency = rate × exposure
# - Total = freq × severity
# - Rolling validation (3 splits)
# ============================================================

import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

def mape(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    mask = y_true != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

# ==========================================
# BUILD MONTHLY FROM STAGE 1 (already clean)
# ==========================================
monthly = monthly.sort_values("year_month").reset_index(drop=True)

monthly["claim_rate"] = (
    monthly["frequency"] /
    monthly["exposure"].replace(0, np.nan)
).fillna(0)

monthly["severity"] = (
    monthly["total_claim"] /
    monthly["frequency"].replace(0, np.nan)
).fillna(0)

# ==========================================
# HORIZON
# ==========================================
sample_sub["year"]  = sample_sub["id"].str.split("_").str[0]
sample_sub["month"] = sample_sub["id"].str.split("_").str[1]
sample_sub["month_key"] = sample_sub["year"] + "-" + sample_sub["month"]

future_periods = (
    pd.PeriodIndex(sample_sub["month_key"], freq="M")
      .unique()
      .sort_values()
)

H = len(future_periods)
N = len(monthly)

# ==========================================
# SIMULATOR
# ==========================================
def simulate(train_df, H, w_rate, w_sev):

    sim = train_df.copy()
    predF, predT, predS = [], [], []

    for step in range(H):

        hist = sim.copy()

        # ----- CLAIM RATE MODEL -----
        try:
            mdl_r = ExponentialSmoothing(
                np.log1p(hist["claim_rate"].astype(float)),
                trend="add",
                damped_trend=True,
                seasonal=None
            ).fit()
            rate_fc = float(np.expm1(mdl_r.forecast(1).iloc[0]))
        except:
            rate_fc = float(hist["claim_rate"].iloc[-1])

        rate_anchor = float(hist["claim_rate"].tail(3).median())
        rate_pred = w_rate*rate_fc + (1-w_rate)*rate_anchor
        rate_pred = max(rate_pred, 1e-9)

        expo_next = float(hist["exposure"].iloc[-1])

        freq_pred = rate_pred * expo_next
        freq_pred = max(freq_pred, 1.0)

        # ----- SEVERITY MODEL -----
        try:
            mdl_s = ExponentialSmoothing(
                np.log1p(hist["severity"].astype(float)),
                trend="add",
                damped_trend=True,
                seasonal=None
            ).fit()
            sev_fc = float(np.expm1(mdl_s.forecast(1).iloc[0]))
        except:
            sev_fc = float(hist["severity"].iloc[-1])

        sev_anchor = float(hist["severity"].tail(3).median())
        sev_pred = w_sev*sev_fc + (1-w_sev)*sev_anchor
        sev_pred = max(sev_pred, 1e-6)

        total_pred = freq_pred * sev_pred

        predF.append(freq_pred)
        predT.append(total_pred)
        predS.append(sev_pred)

        sim = pd.concat([sim, pd.DataFrame([{
            "year_month": hist["year_month"].iloc[-1] + 1,
            "exposure": expo_next,
            "claim_rate": rate_pred,
            "severity": sev_pred,
            "frequency": freq_pred,
            "total_claim": total_pred
        }])], ignore_index=True)

    return predF, predT, predS

# ==========================================
# ROLLING VALIDATION (3 splits)
# ==========================================
splits = [N-H-2, N-H-1, N-H]  # rolling ends
splits = [s for s in splits if s > 6]

w_rate_grid = [0.3,0.5,0.7,0.85]
w_sev_grid  = [0.3,0.5,0.7,0.85]

best = {"score":1e18,"params":None}

for wr in w_rate_grid:
    for ws in w_sev_grid:

        scores = []

        for split in splits:

            train = monthly.iloc[:split]
            valid = monthly.iloc[split:split+H]

            pf, pt, ps = simulate(train,H,wr,ws)

            mf = mape(valid["frequency"],pf)
            mt = mape(valid["total_claim"],pt)
            ms = mape(valid["severity"],ps)

            scores.append(np.nanmean([mf,mt,ms]))

        avg = np.nanmean(scores)

        if avg < best["score"]:
            best["score"] = avg
            best["params"] = (wr,ws)

wr,ws = best["params"]

print("\n==============================")
print("Best Weights:")
print("w_rate:",wr,"w_sev:",ws)
print("Estimated CV Score:",round(best["score"],4))
print("==============================")


Horizon months used : 5
Best Config:
  wt_total=0.85 (ETS weight), anchor_total=median
  wt_freq =0.2 (ETS weight), anchor_freq =mean
------------------------------
STAGE 3 v17 MAPE Frequency : 5.1557
STAGE 3 v17 MAPE Total     : 7.9753
STAGE 3 v17 MAPE Severity  : 4.7684
Estimated Score            : 5.9665

Preview last horizon months:
   year_month  frequency   total_claim      severity  pred_frequency  \
14    2025-03        230  1.367924e+10  5.947496e+07      234.031716   
15    2025-04        208  1.116425e+10  5.367427e+07      232.851773   
16    2025-05        239  1.222680e+10  5.115814e+07      237.225688   
17    2025-06        234  1.337312e+10  5.715008e+07      234.888808   
18    2025-07        264  1.369923e+10  5.189101e+07      235.077202   

      pred_total  pred_severity  
14  1.224504e+10   5.232214e+07  
15  1.224868e+10   5.260289e+07  
16  1.222798e+10   5.154577e+07  
17  1.221086e+10   5.198572e+07  
18  1.219531e+10   5.187790e+07  


# TOTAL CLAIM OPTIMIZATION & VALIDATION, OPTUNA

In [6]:
!pip install -q optuna statsmodels

import numpy as np
import pandas as pd
import optuna
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

SEED = 42
np.random.seed(SEED)

# =============================
# MONTHLY (from Stage 1)
# =============================
monthly = monthly.sort_values("year_month").reset_index(drop=True)

monthly["claim_rate"] = (
    monthly["frequency"] /
    monthly["exposure"].replace(0, np.nan)
).fillna(0)

monthly["severity"] = (
    monthly["total_claim"] /
    monthly["frequency"].replace(0, np.nan)
).fillna(0)

monthly["month"] = monthly["year_month"].dt.month

N = len(monthly)

# =============================
# Horizon
# =============================
sample_sub["year"]  = sample_sub["id"].str.split("_").str[0]
sample_sub["month"] = sample_sub["id"].str.split("_").str[1]
sample_sub["month_key"] = sample_sub["year"] + "-" + sample_sub["month"]

future_periods = (
    pd.PeriodIndex(sample_sub["month_key"], freq="M")
      .unique()
      .sort_values()
)

H = len(future_periods)

# =============================
# Metrics
# =============================
def mape_frac(y_true, y_pred):
    y_true = np.asarray(y_true, float)
    y_pred = np.asarray(y_pred, float)
    mask = y_true != 0
    if mask.sum() == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask]-y_pred[mask])/y_true[mask]))

def avg_mape(yF,pF,yT,pT):
    yS = yT/np.clip(yF,1.0,np.inf)
    pS = pT/np.clip(pF,1.0,np.inf)
    mf = mape_frac(yF,pF)
    mt = mape_frac(yT,pT)
    ms = mape_frac(yS,pS)
    return np.nanmean([mf,mt,ms])

# =============================
# Season builder (TRAIN ONLY)
# =============================
def build_season_log(series, months):
    y = np.log1p(np.asarray(series,float))
    base = np.median(y)
    s = np.zeros(13)
    for m in range(1,13):
        vals = y[months==m]
        if len(vals)>1:
            s[m] = np.median(vals)-base
        else:
            s[m] = 0
    return s

# =============================
# Helpers
# =============================
def ets1(series_log):
    if len(series_log)<4:
        return series_log[-1]
    try:
        m = ExponentialSmoothing(
            series_log,
            trend="add",
            damped_trend=True,
            seasonal=None,
            initialization_method="estimated"
        ).fit()
        return float(m.forecast(1)[0])
    except:
        return float(series_log[-1])

def gated_drift(series_log,k=3):
    if len(series_log)<k+2:
        return series_log[-1]
    y = np.asarray(series_log,float)
    d1 = np.mean(np.diff(y[-(k+1):]))
    d2 = np.mean(np.diff(y[-(k+3):-2]))
    if np.sign(d1)==np.sign(d2):
        return y[-1]+d1
    else:
        return y[-1]

def anchor(series_log,k=3):
    return float(np.median(series_log[-k:]))

# =============================
# Simulator
# =============================
def simulate(train_df,H,P):

    sim = train_df.copy().reset_index(drop=True)

    # build season from TRAIN ONLY
    S_R = build_season_log(sim["claim_rate"].values,sim["month"].values)
    S_S = build_season_log(sim["severity"].values,sim["month"].values)

    pF,pT = [],[]

    for step in range(H):

        next_period = sim["year_month"].iloc[-1]+1
        m_next = int(next_period.month)
        expo_next = float(sim["exposure"].iloc[-1])

        logR = np.log1p(sim["claim_rate"].values)
        logS = np.log1p(sim["severity"].values)

        # deseason
        logR_ds = logR - P["season_scale"]*S_R[sim["month"].values]
        logS_ds = logS - P["season_scale"]*S_S[sim["month"].values]

        # forecast
        r_ets = ets1(logR_ds)
        r_drift = gated_drift(logR_ds,k=P["drift_k"])
        r_anchor = anchor(logR_ds,k=P["k_anchor"])

        s_ets = ets1(logS_ds)
        s_drift = gated_drift(logS_ds,k=P["drift_k"])
        s_anchor = anchor(logS_ds,k=P["k_anchor"])

        r_ds = (
            P["w_ets"]*r_ets +
            P["w_drift"]*r_drift +
            (1-P["w_ets"]-P["w_drift"])*r_anchor
        )

        s_ds = (
            P["w_ets"]*s_ets +
            P["w_drift"]*s_drift +
            (1-P["w_ets"]-P["w_drift"])*s_anchor
        )

        # reseason
        logR_pred = r_ds + P["season_scale"]*S_R[m_next]
        logS_pred = s_ds + P["season_scale"]*S_S[m_next]

        rate_pred = max(np.expm1(logR_pred),1e-9)
        sev_pred  = max(np.expm1(logS_pred),1e-9)

        freq_pred = rate_pred*expo_next
        total_pred = freq_pred*sev_pred

        # clamp via quantile band
        qF_low,qF_high = np.quantile(sim["frequency"],[0.1,0.9])
        qT_low,qT_high = np.quantile(sim["total_claim"],[0.1,0.9])

        freq_pred = float(np.clip(freq_pred,qF_low*0.7,qF_high*1.4))
        total_pred = float(np.clip(total_pred,qT_low*0.7,qT_high*1.4))

        pF.append(freq_pred)
        pT.append(total_pred)

        sim = pd.concat([sim,pd.DataFrame([{
            "year_month":next_period,
            "month":m_next,
            "exposure":expo_next,
            "claim_rate":rate_pred,
            "severity":sev_pred,
            "frequency":freq_pred,
            "total_claim":total_pred
        }])],ignore_index=True)

    return np.array(pF),np.array(pT)

# =============================
# Rolling CV (3 splits)
# =============================
splits = [N-H-2,N-H-1,N-H]
splits = [s for s in splits if s>6]

def run_cv(P):
    scores=[]
    for te in splits:
        tr = monthly.iloc[:te]
        va = monthly.iloc[te:te+H]
        pF,pT = simulate(tr,H,P)
        scores.append(avg_mape(
            va["frequency"].values,pF,
            va["total_claim"].values,pT
        ))
    return np.mean(scores)

# =============================
# Baseline Params
# =============================
P0 = dict(
    w_ets=0.6,
    w_drift=0.2,
    k_anchor=4,
    drift_k=3,
    season_scale=0.5
)

base = run_cv(P0)
print("Baseline CV:",round(base*100,4))

# =============================
# Optuna
# =============================
def objective(trial):
    P=dict(
        w_ets=trial.suggest_float("w_ets",0.3,0.8),
        w_drift=trial.suggest_float("w_drift",0.0,0.4),
        k_anchor=trial.suggest_int("k_anchor",3,6),
        drift_k=trial.suggest_int("drift_k",2,5),
        season_scale=trial.suggest_float("season_scale",0.0,0.8)
    )
    if P["w_ets"]+P["w_drift"]>0.9:
        return 1e9
    return run_cv(P)

study = optuna.create_study(direction="minimize")
study.optimize(objective,n_trials=120,show_progress_bar=True)

bestP = study.best_params
best_score = run_cv(bestP)

print("\n==============================")
print("Best Params:",bestP)
print("CV Best % :",round(best_score*100,4))
print("==============================")
print("STAGE 4 v31 — READY (RATE+SEV, NO LEAK)")

N months: 19 | Horizon H: 5 | Future MOY: [8, 9, 10, 11, 12]
CV train_ends: [7, 8, 13, 14] | weights: [0.636, 0.277, 0.042, 0.045]
  split te= 7 | valid months: ['2024-08', '2024-09', '2024-10', '2024-11', '2024-12']
  split te= 8 | valid months: ['2024-09', '2024-10', '2024-11', '2024-12', '2025-01']
  split te= 13 | valid months: ['2025-02', '2025-03', '2025-04', '2025-05', '2025-06']
  split te= 14 | valid months: ['2025-03', '2025-04', '2025-05', '2025-06', '2025-07']
Baseline CV %: 3.2595
Baseline per-split (%):
  te=7 | avg=1.588 | total=1.175 | freq=2.059 | sev=1.530
  te=8 | avg=6.439 | total=9.479 | freq=4.847 | sev=4.992
  te=13 | avg=6.772 | total=8.802 | freq=8.416 | sev=3.098
  te=14 | avg=4.022 | total=2.204 | freq=5.524 | sev=4.338


  0%|          | 0/200 [00:00<?, ?it/s]


Splits: [7, 8, 13, 14] | weights: [0.636, 0.277, 0.042, 0.045] | overlap: [1.0, 0.8, 0.0, 0.0]
Best Params: {'k_anchor': 4, 'anchor_f': 'mean', 'anchor_t': 'median', 'w_ets_f': 0.4140902012641963, 'w_drift_f': 0.23427959162969844, 'w_ets_t': 0.1253855830888168, 'w_drift_t': 0.0557421860395439, 'drift_k_f': 5, 'drift_k_t': 5, 'season_scale_f': 1.007823739519456, 'season_scale_t': 0.9473922819889735, 'capF_low': 0.8025030597351724, 'capF_high': 1.4449152050189749, 'capT_low': 0.5583495487333925, 'capT_high': 1.7549317144514707}
CV Best %  : 2.7153
Best per-split (%):
  te=7 | avg=1.006 | total=0.853 | freq=1.516 | sev=0.651
  te=8 | avg=5.992 | total=9.223 | freq=4.088 | sev=4.666
  te=13 | avg=5.574 | total=6.910 | freq=7.950 | sev=1.862
  te=14 | avg=4.018 | total=2.471 | freq=4.810 | sev=4.773

STAGE 4 v30 — READY (global season prior)


# TEST PREDICTION & KAGGLE SUBMISSION

In [7]:
# ============================================================
# STAGE 5 v32 — FINAL SUBMISSION (Stage3 + Stage4 v31) ROBUST 2D BLEND
# - Stage3: predict Frequency & Total directly (ETS log1p + anchor shrink)
# - Stage4 v31: predict claim_rate & severity (season built TRAIN-ONLY), derive F/T
# - Blend (regularized):
#     F = (1-wF)*F3 + wF*F4
#     T = (1-wT)*T3 + wT*T4
#   + safety: clamp blended outputs to stay near Stage3/Stage4 corridor
# - Objective: weighted mean + lambda_worst*worst + lambda_dev*deviation
# ============================================================

!pip install -q statsmodels

import numpy as np, pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

SEED = 42
np.random.seed(SEED)

BASE_PATH = "/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/"
sample_sub = pd.read_csv(BASE_PATH + "sample_submission.csv")

# ------------------------------
# 0) SAFETY CHECKS
# ------------------------------
assert "monthly" in globals(), "monthly belum ada. Jalankan Stage 1."
assert "year_month" in monthly.columns, "monthly['year_month'] tidak ada."
assert "exposure" in monthly.columns, "monthly['exposure'] tidak ada. Stage 1 exposure wajib."
assert "frequency" in monthly.columns and "total_claim" in monthly.columns, "monthly freq/total wajib."

# ------------------------------
# 1) FUTURE PERIODS + H
# ------------------------------
sample_sub["year"]  = sample_sub["id"].str.split("_").str[0]
sample_sub["month"] = sample_sub["id"].str.split("_").str[1]
sample_sub["month_key"] = sample_sub["year"] + "-" + sample_sub["month"]

future_periods = pd.PeriodIndex(sample_sub["month_key"], freq="M").unique().sort_values()
H0 = int(len(future_periods))
future_moy = set([p.month for p in future_periods])

# ------------------------------
# 2) MONTHLY COMPLETE RANGE + CLEAN
# ------------------------------
m = monthly.copy().sort_values("year_month").reset_index(drop=True)

if not isinstance(m.loc[0, "year_month"], pd.Period):
    m["year_month"] = pd.PeriodIndex(m["year_month"], freq="M")

min_m = m["year_month"].min()
max_m = m["year_month"].max()
all_months = pd.period_range(min_m, max_m, freq="M")

m = (
    m.set_index("year_month")
     .reindex(all_months)
     .rename_axis("year_month")
     .reset_index()
)

# keep exposure last-known forward fill (portfolio often stable)
if "exposure" in m.columns:
    m["exposure"] = m["exposure"].ffill().bfill()

m["frequency"]   = pd.to_numeric(m["frequency"], errors="coerce").fillna(0.0).clip(lower=1.0)
m["total_claim"] = pd.to_numeric(m["total_claim"], errors="coerce").fillna(0.0).clip(lower=1.0)
m["severity"]    = (m["total_claim"] / m["frequency"]).astype(float)
m["month"]       = m["year_month"].dt.month
m["t"]           = np.arange(len(m), dtype=int)

# rate & sev for Stage4 v31
m["claim_rate"] = (m["frequency"] / m["exposure"].replace(0, np.nan)).fillna(0.0)
m["severity"]   = (m["total_claim"] / m["frequency"].replace(0, np.nan)).fillna(0.0)

N = len(m)
H = min(H0, max(1, N - 10))
print("N months:", N, "| H:", H, "| Future MOY:", sorted(list(future_moy)))

# ------------------------------
# 3) METRICS
# ------------------------------
def mape_frac(y_true, y_pred):
    y_true = np.asarray(y_true, float)
    y_pred = np.asarray(y_pred, float)
    mask = np.isfinite(y_true) & np.isfinite(y_pred) & (y_true != 0)
    if mask.sum() == 0:
        return np.nan
    return float(np.mean(np.abs((y_true[mask]-y_pred[mask]) / y_true[mask])))

def avg_mape(yF, pF, yT, pT):
    yF = np.asarray(yF, float); pF = np.asarray(pF, float)
    yT = np.asarray(yT, float); pT = np.asarray(pT, float)
    yS = yT / np.clip(yF, 1.0, np.inf)
    pS = pT / np.clip(pF, 1.0, np.inf)
    mf = mape_frac(yF, pF)
    mt = mape_frac(yT, pT)
    ms = mape_frac(yS, pS)
    return float(np.nanmean([mf, mt, ms])), mt, mf, ms

# ------------------------------
# 4) STAGE 3 SIMULATOR (FROM YOUR STAGE 3)
# ------------------------------
# Use your tuned S3 (or keep these defaults)
S3 = dict(
    wt_total=0.85, anchor_total="median",
    wt_freq=0.20,  anchor_freq="mean",
    k_anchor=3
)

def anchor_level_series(x: pd.Series, k: int, how: str):
    tail = np.asarray(x.tail(k), dtype=float)
    return float(np.median(tail)) if how == "median" else float(np.mean(tail))

def ets_1step_log1p_series(level_series: pd.Series, trend="add", damped=True):
    y = np.log1p(level_series.astype(float).clip(lower=1e-12))
    if len(y) < 4:
        return float(np.expm1(y.iloc[-1]))
    if trend is not None and len(y) < 10:
        trend = None
        damped = False
    try:
        mdl = ExponentialSmoothing(
            y, trend=trend,
            damped_trend=(damped if trend is not None else False),
            seasonal=None,
            initialization_method="estimated"
        ).fit()
        return float(np.expm1(mdl.forecast(1).iloc[0]))
    except:
        return float(level_series.iloc[-1])

def simulate_stage3(train_df: pd.DataFrame, H: int):
    sim = train_df.copy().reset_index(drop=True)
    pF, pT = [], []
    for _ in range(H):
        k = int(S3["k_anchor"])

        tot_fc = ets_1step_log1p_series(sim["total_claim"], trend="add", damped=True)
        tot_anchor = anchor_level_series(sim["total_claim"], k, S3["anchor_total"])
        tot_pred = float(S3["wt_total"])*tot_fc + (1-float(S3["wt_total"]))*tot_anchor
        tot_pred = max(1.0, tot_pred)

        fre_fc = ets_1step_log1p_series(sim["frequency"], trend="add", damped=True)
        fre_anchor = anchor_level_series(sim["frequency"], k, S3["anchor_freq"])
        fre_pred = float(S3["wt_freq"])*fre_fc + (1-float(S3["wt_freq"]))*fre_anchor
        fre_pred = max(1.0, fre_pred)

        pF.append(fre_pred)
        pT.append(tot_pred)

        sim = pd.concat([sim, pd.DataFrame([{
            "year_month": sim["year_month"].iloc[-1] + 1,
            "month": int((sim["year_month"].iloc[-1] + 1).month),
            "t": int(sim["t"].iloc[-1] + 1),
            "exposure": float(sim["exposure"].iloc[-1]),
            "frequency": fre_pred,
            "total_claim": tot_pred,
            "claim_rate": float(fre_pred / max(1.0, float(sim["exposure"].iloc[-1]))),
            "severity": float(tot_pred / max(1.0, fre_pred)),
        }])], ignore_index=True)

    return np.array(pF, float), np.array(pT, float)

# ------------------------------
# 5) STAGE 4 v31 SIMULATOR (RATE+SEV TRAIN-ONLY SEASON)
# ------------------------------
# Use bestP from your Stage4 v31 output. If not exists, fallback baseline.
if "bestP" in globals():
    P4 = dict(bestP)
else:
    P4 = dict(w_ets=0.6, w_drift=0.2, k_anchor=4, drift_k=3, season_scale=0.5)

def build_season_log(series, months):
    y = np.log1p(np.asarray(series, float).clip(min=0))
    base = float(np.median(y))
    s = np.zeros(13, dtype=float)
    for mm in range(1, 13):
        vals = y[np.asarray(months, int) == mm]
        s[mm] = float(np.median(vals) - base) if len(vals) > 1 else 0.0
    return s

def ets1(series_log):
    y = np.asarray(series_log, float)
    if len(y) < 4:
        return float(y[-1])
    try:
        mdl = ExponentialSmoothing(
            y,
            trend="add",
            damped_trend=True,
            seasonal=None,
            initialization_method="estimated"
        ).fit()
        return float(mdl.forecast(1)[0])
    except:
        return float(y[-1])

def gated_drift(series_log, k=3):
    y = np.asarray(series_log, float)
    if len(y) < (k + 2):
        return float(y[-1])
    d1 = float(np.mean(np.diff(y[-(k+1):])))
    d2 = float(np.mean(np.diff(y[-(k+3):-2]))) if len(y) >= (k+4) else d1
    return float(y[-1] + d1) if np.sign(d1) == np.sign(d2) else float(y[-1])

def anchor_log(series_log, k=3):
    y = np.asarray(series_log, float)
    return float(np.median(y[-k:]))

def simulate_stage4v31(train_df: pd.DataFrame, H: int):
    sim = train_df.copy().reset_index(drop=True)

    # season from TRAIN ONLY
    SR = build_season_log(sim["claim_rate"].values, sim["month"].values)
    SS = build_season_log(sim["severity"].values, sim["month"].values)

    pF, pT = [], []
    for _ in range(H):
        next_period = sim["year_month"].iloc[-1] + 1
        m_next = int(next_period.month)

        expo_next = float(sim["exposure"].iloc[-1])
        expo_next = max(1.0, expo_next)

        logR = np.log1p(sim["claim_rate"].values.clip(min=0))
        logS = np.log1p(sim["severity"].values.clip(min=0))

        # deseason
        sc = float(P4.get("season_scale", 0.5))
        logR_ds = logR - sc * SR[sim["month"].values]
        logS_ds = logS - sc * SS[sim["month"].values]

        # components
        r_ets = ets1(logR_ds)
        r_dft = gated_drift(logR_ds, k=int(P4.get("drift_k", 3)))
        r_an  = anchor_log(logR_ds, k=int(P4.get("k_anchor", 4)))

        s_ets = ets1(logS_ds)
        s_dft = gated_drift(logS_ds, k=int(P4.get("drift_k", 3)))
        s_an  = anchor_log(logS_ds, k=int(P4.get("k_anchor", 4)))

        wE = float(P4.get("w_ets", 0.6))
        wD = float(P4.get("w_drift", 0.2))
        if (wE + wD) > 0.9:
            # keep convex
            wD = max(0.0, 0.9 - wE)

        r_ds = wE*r_ets + wD*r_dft + (1.0-wE-wD)*r_an
        s_ds = wE*s_ets + wD*s_dft + (1.0-wE-wD)*s_an

        # reseason
        logR_pred = float(r_ds + sc * SR[m_next])
        logS_pred = float(s_ds + sc * SS[m_next])

        rate_pred = float(max(np.expm1(logR_pred), 1e-9))
        sev_pred  = float(max(np.expm1(logS_pred), 1e-9))

        F_pred = float(rate_pred * expo_next)
        T_pred = float(F_pred * sev_pred)

        # light clamp vs training quantiles (anti spike)
        qF_low, qF_high = np.quantile(sim["frequency"].values, [0.10, 0.90])
        qT_low, qT_high = np.quantile(sim["total_claim"].values, [0.10, 0.90])

        F_pred = float(np.clip(F_pred, max(1.0, qF_low*0.70), qF_high*1.40))
        T_pred = float(np.clip(T_pred, max(1.0, qT_low*0.70), qT_high*1.40))

        pF.append(F_pred)
        pT.append(T_pred)

        sim = pd.concat([sim, pd.DataFrame([{
            "year_month": next_period,
            "month": m_next,
            "t": int(sim["t"].iloc[-1] + 1),
            "exposure": expo_next,
            "claim_rate": rate_pred,
            "severity": sev_pred,
            "frequency": F_pred,
            "total_claim": T_pred,
        }])], ignore_index=True)

    return np.array(pF, float), np.array(pT, float)

# ------------------------------
# 6) CV SPLITS (ROBUST, NO FIXED HAND-PICK)
# ------------------------------
# Rolling splits near the end + overlap-aware weighting
min_train = 7
cand_te = list(range(min_train, N - H + 1))
# Keep only last ~6 splits to be "Kaggle-like"
cand_te = cand_te[-6:] if len(cand_te) > 6 else cand_te

def overlap_ratio(te):
    v = m.iloc[te:te+H]
    return sum([1 for mm in v["month"].tolist() if mm in future_moy]) / float(H)

ovs = np.array([overlap_ratio(te) for te in cand_te], float)
rcs = np.array([te/float(N) for te in cand_te], float)

# weights: emphasize overlap + some recency
OVERLAP_POWER = 3.0
RECENCY_MIX = 0.20
w_raw = (ovs ** OVERLAP_POWER) + RECENCY_MIX * rcs
w_raw = np.maximum(w_raw, 1e-6)
split_w = w_raw / w_raw.sum()

train_ends = cand_te
print("CV train_ends:", train_ends)
print("overlap:", ovs.round(3).tolist())
print("weights:", split_w.round(3).tolist())

# ------------------------------
# 7) BLEND SEARCH (REGULARIZED)
# ------------------------------
grid = np.linspace(0.0, 1.0, 21)  # step 0.05

# penalties to prevent LB collapse
LAMBDA_WORST = 0.35
LAMBDA_DEV   = 0.10

def corridor_clip(x, a, b, lo=0.85, hi=1.15):
    # keep within both model corridors:
    # - near Stage3: [lo*a, hi*a]
    # - near Stage4: [lo*b, hi*b]
    low = np.minimum(lo*a, lo*b)
    high = np.maximum(hi*a, hi*b)
    return np.clip(x, low, high)

best = None
rows = []

for wF in grid:
    for wT in grid:
        split_scores = []
        dev_scores = []
        for te, w in zip(train_ends, split_w):
            tr = m.iloc[:te].copy().reset_index(drop=True)
            va = m.iloc[te:te+H].copy().reset_index(drop=True)

            F3, T3 = simulate_stage3(tr, H)
            F4, T4 = simulate_stage4v31(tr, H)

            F = (1-wF)*F3 + wF*F4
            T = (1-wT)*T3 + wT*T4

            # safety corridor clip (anti out-of-distribution)
            F = corridor_clip(F, F3, F4, lo=0.88, hi=1.12)
            T = corridor_clip(T, T3, T4, lo=0.88, hi=1.12)

            sc, mt, mf, ms = avg_mape(va["frequency"].values, F, va["total_claim"].values, T)
            split_scores.append(sc)

            # deviation penalty from Stage3 (keep anchor stability)
            dev = float(np.mean(np.abs(F/F3 - 1.0)) + np.mean(np.abs(T/T3 - 1.0)))
            dev_scores.append(dev)

        meanw = float(np.sum(split_w * np.array(split_scores)))
        worst = float(np.max(split_scores))
        devw  = float(np.sum(split_w * np.array(dev_scores)))

        obj = meanw + LAMBDA_WORST*worst + LAMBDA_DEV*devw

        rows.append([wF, wT, obj, meanw, worst, devw] + split_scores)

        cand = (obj, meanw, worst, devw, wF, wT)
        if (best is None) or (cand < best):
            best = cand

obj_best, mean_best, worst_best, dev_best, wF_best, wT_best = best

print("\nBest blend (REG):")
print("  wF_best =", wF_best, "| wT_best =", wT_best)
print("  obj     =", round(obj_best*100, 4), "%")
print("  mean    =", round(mean_best*100, 4), "% | worst =", round(worst_best*100, 4), "% | dev =", round(dev_best, 4))

dfb = pd.DataFrame(rows, columns=["wF","wT","obj","mean_avg","worst_avg","dev"] + [f"split_{i}" for i in range(len(train_ends))])
print("\nTop 10 by obj:")
print(dfb.sort_values(["obj","worst_avg","dev"]).head(10))

# ------------------------------
# 8) FINAL FORECAST ON FULL DATA
# ------------------------------
F3, T3 = simulate_stage3(m, H)
F4, T4 = simulate_stage4v31(m, H)

F = (1-wF_best)*F3 + wF_best*F4
T = (1-wT_best)*T3 + wT_best*T4

# final safety corridor (important)
F = corridor_clip(F, F3, F4, lo=0.88, hi=1.12)
T = corridor_clip(T, T3, T4, lo=0.88, hi=1.12)

S = T / np.clip(F, 1.0, np.inf)

pred_map = {}
preview = []
for i, period in enumerate(future_periods):
    key = f"{period.year}_{str(period.month).zfill(2)}"
    pred_map[f"{key}_Claim_Frequency"] = float(F[i])
    pred_map[f"{key}_Total_Claim"]     = float(T[i])
    pred_map[f"{key}_Claim_Severity"]  = float(S[i])
    preview.append([str(period), float(F[i]), float(T[i]), float(S[i])])

sub = sample_sub.copy()
sub["value"] = sub["id"].map(pred_map)
missing = int(sub["value"].isna().sum())
print("\nNaN in submission:", missing)
assert missing == 0, "Ada id belum terisi. Cek format key."

sub = sub[["id","value"]]
sub.to_csv("submission.csv", index=False)

print("\nPreview future predictions:")
print(pd.DataFrame(preview, columns=["period","pred_freq","pred_total","pred_sev"]))
print("\nSaved: submission.csv")
print(sub.head(12))

N months: 19 | H: 5 | Future MOY: [8, 9, 10, 11, 12]
Blend train_ends: [7, 8, 13, 14] | weights: [0.636, 0.277, 0.042, 0.045]

Best blend (2D):
  wF_best = 1.0 (Stage4 weight for Frequency)
  wT_best = 1.0 (Stage4 weight for Total)
  CV mean = 2.7148 % | worst = 5.9924 %

Top 10 by mean_avg:
       wF    wT  mean_avg  worst_avg   split_0   split_1   split_2   split_3
440  1.00  1.00  0.027148   0.059924  0.010063  0.059924  0.055742  0.040181
419  0.95  1.00  0.027608   0.060917  0.010501  0.060917  0.053238  0.040423
439  1.00  0.95  0.027650   0.060254  0.010683  0.060254  0.056984  0.039366
418  0.95  0.95  0.028299   0.060834  0.011568  0.060834  0.054961  0.039608
398  0.90  1.00  0.029244   0.062815  0.012304  0.062815  0.052115  0.040664
417  0.95  0.90  0.029478   0.061349  0.013117  0.061349  0.057053  0.038792
438  1.00  0.90  0.029507   0.061363  0.013037  0.061363  0.059136  0.038551
397  0.90  0.95  0.029595   0.062734  0.012836  0.062734  0.053834  0.039848
396  0.90  0.9

In [8]:
print(sub.head(12)) ##  ini yang stage 4 diganti

                         id         value
0   2025_08_Claim_Frequency  2.232146e+02
1    2025_08_Claim_Severity  5.929055e+07
2       2025_08_Total_Claim  1.323452e+10
3   2025_09_Claim_Frequency  2.041827e+02
4    2025_09_Claim_Severity  5.933691e+07
5       2025_09_Total_Claim  1.211557e+10
6   2025_10_Claim_Frequency  2.703015e+02
7    2025_10_Claim_Severity  4.604846e+07
8       2025_10_Total_Claim  1.244697e+10
9   2025_11_Claim_Frequency  2.655660e+02
10   2025_11_Claim_Severity  5.043058e+07
11      2025_11_Total_Claim  1.339265e+10


In [10]:
# ============================================================
# DIAG CELL — build 3 submissions:
# 1) sub_stage3.csv
# 2) sub_stage4v30.csv
# 3) sub_blend_safe.csv (regularized blend, won't drift too far from Stage3)
# ============================================================

import numpy as np, pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

BASE_PATH = "/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/"
sample_sub = pd.read_csv(BASE_PATH + "sample_submission.csv")

assert "df" in globals()
assert "year_month" in df.columns

# ---- future periods ----
sample_sub["year"]  = sample_sub["id"].str.split("_").str[0]
sample_sub["month"] = sample_sub["id"].str.split("_").str[1]
sample_sub["month_key"] = sample_sub["year"] + "-" + sample_sub["month"]
future_periods = pd.PeriodIndex(sample_sub["month_key"], freq="M").unique().sort_values()
H = int(len(future_periods))

# ---- monthly portfolio ----
monthly = (
    df.groupby("year_month")
      .agg(frequency=("claim_id","count"),
           total_claim=("nominal_klaim_yang_disetujui","sum"))
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)
if not isinstance(monthly.loc[0,"year_month"], pd.Period):
    monthly["year_month"] = pd.PeriodIndex(monthly["year_month"], freq="M")

min_m = monthly["year_month"].min()
max_m = monthly["year_month"].max()
all_months = pd.period_range(min_m, max_m, freq="M")

monthly = (monthly.set_index("year_month")
                 .reindex(all_months, fill_value=0.0)
                 .rename_axis("year_month")
                 .reset_index())
monthly["frequency"]   = pd.to_numeric(monthly["frequency"], errors="coerce").fillna(0.0).clip(lower=1.0)
monthly["total_claim"] = pd.to_numeric(monthly["total_claim"], errors="coerce").fillna(0.0).clip(lower=1.0)
monthly["month"] = monthly["year_month"].dt.month
monthly["t"] = np.arange(len(monthly), dtype=int)

# ============================================================
# Stage 3 simulator (as used before)
# ============================================================
S3 = dict(wt_total=0.85, anchor_total="median", wt_freq=0.20, anchor_freq="mean", k_anchor=3)

def anchor_level(x: pd.Series, k: int, how: str):
    tail = np.asarray(x.tail(k), dtype=float)
    return float(np.median(tail)) if how == "median" else float(np.mean(tail))

def ets_1step_log1p(level_series: pd.Series, trend="add", damped=True):
    y = np.log1p(level_series.astype(float).clip(lower=1e-12))
    if len(y) < 4:
        return float(np.expm1(y.iloc[-1]))
    if trend is not None and len(y) < 10:
        trend = None
        damped = False
    try:
        m = ExponentialSmoothing(
            y, trend=trend,
            damped_trend=(damped if trend is not None else False),
            seasonal=None
        ).fit()
        return float(np.expm1(m.forecast(1).iloc[0]))
    except:
        return float(level_series.iloc[-1])

def simulate_stage3(train_df: pd.DataFrame, H: int):
    sim = train_df.copy().reset_index(drop=True)
    pF, pT = [], []
    for _ in range(H):
        k = int(S3["k_anchor"])
        tot_fc = ets_1step_log1p(sim["total_claim"], trend="add", damped=True)
        tot_anchor = anchor_level(sim["total_claim"], k, S3["anchor_total"])
        tot_pred = float(S3["wt_total"])*tot_fc + (1-float(S3["wt_total"]))*tot_anchor
        tot_pred = max(1.0, tot_pred)

        fre_fc = ets_1step_log1p(sim["frequency"], trend="add", damped=True)
        fre_anchor = anchor_level(sim["frequency"], k, S3["anchor_freq"])
        fre_pred = float(S3["wt_freq"])*fre_fc + (1-float(S3["wt_freq"]))*fre_anchor
        fre_pred = max(1.0, fre_pred)

        pF.append(fre_pred); pT.append(tot_pred)
        sim = pd.concat([sim, pd.DataFrame([{
            "year_month": sim["year_month"].iloc[-1] + 1,
            "month": int((sim["year_month"].iloc[-1] + 1).month),
            "t": int(sim["t"].iloc[-1] + 1),
            "frequency": fre_pred,
            "total_claim": tot_pred
        }])], ignore_index=True)
    return np.array(pF,float), np.array(pT,float)

# ============================================================
# Stage 4 v30 simulator (paste BEST params kamu)
# ============================================================
BEST4 = {
    'k_anchor': 4, 'anchor_f': 'mean', 'anchor_t': 'median',
    'w_ets_f': 0.4140902012641963, 'w_drift_f': 0.23427959162969844,
    'w_ets_t': 0.1253855830888168, 'w_drift_t': 0.0557421860395439,
    'drift_k_f': 5, 'drift_k_t': 5,
    'season_scale_f': 1.007823739519456, 'season_scale_t': 0.9473922819889735,
    'capF_low': 0.8025030597351724, 'capF_high': 1.4449152050189749,
    'capT_low': 0.5583495487333925, 'capT_high': 1.7549317144514707
}

def build_season_log(series_level, months, clip_abs=0.30):
    y = np.log1p(np.asarray(series_level, float).clip(min=0))
    m = np.asarray(months, int)
    base = float(np.median(y))
    s = np.zeros(13, dtype=float)
    for mm in range(1, 13):
        vals = y[m == mm]
        s[mm] = float(np.median(vals) - base) if len(vals) else 0.0
    return np.clip(s, -float(clip_abs), float(clip_abs))

SEASON_LOG_F_FULL = build_season_log(monthly["frequency"].values, monthly["month"].values, clip_abs=0.30)
SEASON_LOG_T_FULL = build_season_log(monthly["total_claim"].values, monthly["month"].values, clip_abs=0.35)

def ets_1step_on_log(series_log):
    y = np.asarray(series_log, float)
    if len(y) < 4:
        return float(y[-1])
    trend = "add" if len(y) >= 10 else None
    damped = True if trend is not None else False
    try:
        m = ExponentialSmoothing(
            y, trend=trend,
            damped_trend=damped if trend is not None else False,
            seasonal=None,
            initialization_method="estimated"
        ).fit()
        return float(m.forecast(1)[0])
    except:
        return float(y[-1])

def drift_on_log(series_log, k=3):
    y = np.asarray(series_log, float)
    if len(y) < (k + 2):
        return float(y[-1])
    deltas = np.diff(y[-(k+1):])
    return float(y[-1] + np.mean(deltas))

def anchor_on_log(series_log, k=3, how="mean"):
    tail = np.asarray(series_log[-k:], float)
    return float(np.median(tail)) if how == "median" else float(np.mean(tail))

def simulate_stage4v30(train_df: pd.DataFrame, H: int):
    P = BEST4
    sim = train_df.copy().reset_index(drop=True)
    sim["logF"] = np.log1p(sim["frequency"].values)
    sim["logT"] = np.log1p(sim["total_claim"].values)
    sf = float(P["season_scale_f"]); st = float(P["season_scale_t"])

    pF, pT = [], []
    for _ in range(H):
        next_period = sim["year_month"].iloc[-1] + 1
        m_next = int(next_period.month)

        logF_ds = sim["logF"].values - sf * SEASON_LOG_F_FULL[sim["month"].values]
        logT_ds = sim["logT"].values - st * SEASON_LOG_T_FULL[sim["month"].values]

        f_ets = ets_1step_on_log(logF_ds)
        f_drift = drift_on_log(logF_ds, k=int(P["drift_k_f"]))
        f_an = anchor_on_log(logF_ds, k=int(P["k_anchor"]), how=P["anchor_f"])
        f_ds_pred = float(P["w_ets_f"])*f_ets + float(P["w_drift_f"])*f_drift + (1-float(P["w_ets_f"])-float(P["w_drift_f"]))*f_an

        t_ets = ets_1step_on_log(logT_ds)
        t_drift = drift_on_log(logT_ds, k=int(P["drift_k_t"]))
        t_an = anchor_on_log(logT_ds, k=int(P["k_anchor"]), how=P["anchor_t"])
        t_ds_pred = float(P["w_ets_t"])*t_ets + float(P["w_drift_t"])*t_drift + (1-float(P["w_ets_t"])-float(P["w_drift_t"]))*t_an

        logF_pred = float(f_ds_pred + sf * SEASON_LOG_F_FULL[m_next])
        logT_pred = float(t_ds_pred + st * SEASON_LOG_T_FULL[m_next])

        F_pred = float(np.expm1(logF_pred))
        T_pred = float(np.expm1(logT_pred))

        # clamp vs anchor
        F_an = float(np.expm1(float(f_an + sf*SEASON_LOG_F_FULL[m_next])))
        T_an = float(np.expm1(float(t_an + st*SEASON_LOG_T_FULL[m_next])))

        F_pred = float(np.clip(F_pred, max(1.0, F_an*float(P["capF_low"])), F_an*float(P["capF_high"])))
        T_pred = float(np.clip(T_pred, max(1.0, T_an*float(P["capT_low"])), T_an*float(P["capT_high"])))

        pF.append(F_pred); pT.append(T_pred)

        sim = pd.concat([sim, pd.DataFrame([{
            "year_month": next_period,
            "month": m_next,
            "t": int(sim["t"].iloc[-1] + 1),
            "frequency": F_pred,
            "total_claim": T_pred,
            "logF": float(np.log1p(F_pred)),
            "logT": float(np.log1p(T_pred)),
        }])], ignore_index=True)

    return np.array(pF,float), np.array(pT,float)

# ============================================================
# Build three forecasts on FULL data
# ============================================================
F3, T3 = simulate_stage3(monthly, H)
F4, T4 = simulate_stage4v30(monthly, H)

# 1) Stage3 only
F_A, T_A = F3, T3
S_A = T_A / np.clip(F_A, 1.0, np.inf)

# 2) Stage4v30 only
F_B, T_B = F4, T4
S_B = T_B / np.clip(F_B, 1.0, np.inf)

# 3) Safe blend: start from stage4 but don't deviate too far from stage3
#    (clip blended ratio to stage3 within +-10% by default)
wF, wT = 1.0, 1.0  # start fully stage4
F_C = (1-wF)*F3 + wF*F4
T_C = (1-wT)*T3 + wT*T4

# clip toward stage3 (safety net)
F_C = np.clip(F_C, F3*0.90, F3*1.10)
T_C = np.clip(T_C, T3*0.90, T3*1.10)
S_C = T_C / np.clip(F_C, 1.0, np.inf)

def make_sub(F, T, S, out_name):
    pred_map = {}
    for i, period in enumerate(future_periods):
        key = f"{period.year}_{str(period.month).zfill(2)}"
        pred_map[f"{key}_Claim_Frequency"] = float(F[i])
        pred_map[f"{key}_Total_Claim"]     = float(T[i])
        pred_map[f"{key}_Claim_Severity"]  = float(S[i])

    sub = sample_sub.copy()
    sub["value"] = sub["id"].map(pred_map)
    assert int(sub["value"].isna().sum()) == 0
    sub = sub[["id","value"]]
    sub.to_csv(out_name, index=False)
    print("Saved:", out_name)
    print(sub.head(6))

make_sub(F_A, T_A, S_A, "sub_stage3.csv")
make_sub(F_B, T_B, S_B, "sub_stage4v30.csv")
make_sub(F_C, T_C, S_C, "sub_blend_safe.csv")

print("\nQuick preview (period, F3,F4,F_safe):")
print(pd.DataFrame({
    "period": [str(p) for p in future_periods],
    "F3": F3, "F4": F4, "F_safe": F_C,
    "T3": T3, "T4": T4, "T_safe": T_C
}))

Saved: sub_stage3.csv
                        id         value
0  2025_08_Claim_Frequency  2.435652e+02
1   2025_08_Claim_Severity  5.161099e+07
2      2025_08_Total_Claim  1.257064e+10
3  2025_09_Claim_Frequency  2.449253e+02
4   2025_09_Claim_Severity  5.133448e+07
5      2025_09_Total_Claim  1.257311e+10
Saved: sub_stage4v30.csv
                        id         value
0  2025_08_Claim_Frequency  2.232146e+02
1   2025_08_Claim_Severity  5.929055e+07
2      2025_08_Total_Claim  1.323452e+10
3  2025_09_Claim_Frequency  2.041827e+02
4   2025_09_Claim_Severity  5.933691e+07
5      2025_09_Total_Claim  1.211557e+10
Saved: sub_blend_safe.csv
                        id         value
0  2025_08_Claim_Frequency  2.232146e+02
1   2025_08_Claim_Severity  5.929055e+07
2      2025_08_Total_Claim  1.323452e+10
3  2025_09_Claim_Frequency  2.204327e+02
4   2025_09_Claim_Severity  5.496266e+07
5      2025_09_Total_Claim  1.211557e+10

Quick preview (period, F3,F4,F_safe):
    period          F3     